# Natural Languange Processing

## Level 1: Apa itu Teks? (The Raw Material)

Di level paling dasar, komputer tidak melihat "kata" atau "kalimat". Ia hanya melihat serangkaian karakter: 
['H', 'e', 'l', 'l', 'o', ' ', 'w', 'o', 'r', 'l', 'd', '!'].


- **Tantangan Utama NLP**: Bagaimana kita mengubah urutan karakter ini menjadi sesuatu yang memiliki makna dan bisa diolah secara matematis?

- **Konsep Kunci:**
  - **Corpus (jamak: Corpora)**: Kumpulan besar dari dokumen teks. Contoh: semua artikel Wikipedia, atau semua ulasan film di IMDb. Ini adalah "bahan baku" kita.
  - **Document**: Satu unit teks dalam corpus. Contoh: satu artikel Wikipedia, atau satu ulasan film.
  - **Token**: Unit dasar dari teks setelah diproses. Biasanya, sebuah "kata", tapi bisa juga tanda baca. Contoh: ["Hello", ",", "world", "!"].
  - **Vocabulary**: Kumpulan semua token unik yang ada di dalam corpus kita.

## Level 2: Membuat Teks Menjadi Terstruktur (Preprocessing)

Sebelum kita bisa mengubah teks menjadi angka, kita harus membersihkan dan menstandarisasinya. Ini adalah tahap "pencucian" dan "pemotongan" bahan baku.

- **Tujuan**: Mengurangi noise dan kompleksitas agar model bisa fokus pada sinyal yang penting.
- **Teknik Umum (Pipeline Preprocessing)**:
  - **Lowercasing**: Mengubah semua teks menjadi huruf kecil. "Hello" dan "hello" dianggap sama. Ini mengurangi ukuran vocabulary.
  - **Tokenization**: Memecah kalimat menjadi token-token (kata). "Hello world!" menjadi ['Hello', 'world', '!']. Ini adalah langkah paling fundamental.
  - **Removing Punctuation & Special Characters**: Menghapus tanda baca seperti ,, . atau karakter seperti #, @.
  - **Removing Stopwords**: Menghapus kata-kata yang sangat umum tetapi seringkali tidak membawa banyak makna, seperti "dan", "di", "yang", "a", "the", "is".
  - **Stemming & Lemmatization**: Mengurangi kata ke bentuk dasarnya.
    - **Stemming**: Proses kasar memotong akhir kata. Contoh: computing, computer, computes -> comput. Cepat tapi kadang hasilnya bukan kata yang valid.
    - **Lemmatization**: Proses yang lebih canggih menggunakan kamus untuk mengubah kata ke bentuk dasarnya (lemma). Contoh: am, are, is -> be; better -> good. Lebih lambat tapi lebih akurat.

**Contoh Pipeline:**

- **Input**: "Wow, the computers are learning so fast!"
- **Lowercase**: "wow, the computers are learning so fast!"
- **Tokenize**: ['wow', ',', 'the', 'computers', 'are', 'learning', 'so', 'fast', '!']
- **Remove Punctuation**: ['wow', 'the', 'computers', 'are', 'learning', 'so', 'fast']
- **Remove Stopwords**: ['wow', 'computers', 'learning', 'fast']
- **Lemmatize**: ['wow', 'computer', 'learn', 'fast']
- **Output Siap Proses**: ['wow', 'computer', 'learn', 'fast']

In [1]:
import nltk
# Unduh paket-paket yang diperlukan (hanya perlu sekali)
nltk.download('punkt')      # Untuk tokenisasi
nltk.download('stopwords')  # Untuk daftar stopwords
nltk.download('wordnet')    # Untuk lemmatisasi
nltk.download('omw-1.4')    # Paket tambahan untuk WordNet

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mulin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mulin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mulin\AppData\Roaming\nltk_data...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\mulin\AppData\Roaming\nltk_data...


True

In [2]:
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

def preprocess_text(text):
    """
    Membersihkan dan memproses satu string teks.
    
    Langkah-langkah:
    1. Mengubah ke huruf kecil
    2. Menghapus URL, handle @, dan karakter non-alfanumerik
    3. Tokenisasi
    4. Menghapus stopwords
    5. Lemmatisasi
    
    Args:
        text (str): String teks mentah.
        
    Returns:
        list: Daftar token yang sudah dibersihkan.
    """
    # 1. Lowercasing
    text = text.lower()
    
    # 2. Menghapus elemen-elemen spesifik (URL, handle, hashtag, dll.)
    # Hapus URL
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    # Hapus handle @ dan hashtag #
    text = re.sub(r'@[^\s]+|#[^\s]+', '', text)
    # Hapus karakter non-alfabet (menyisakan spasi)
    text = re.sub(r'[^a-z\s]', '', text)
    
    # 3. Tokenisasi
    tokens = word_tokenize(text)
    
    # 4. Menghapus stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]
    
    # 5. Lemmatisasi
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in filtered_tokens]
    
    return lemmatized_tokens

# --- Contoh Penggunaan ---
raw_text = "Wow, the computers at https://example.com are learning so fast! @ML_Lover #AI"
cleaned_tokens = preprocess_text(raw_text)

print(f"Teks Mentah:\n'{raw_text}'")
print(f"\nToken Bersih:\n{cleaned_tokens}")
# Output yang diharapkan: ['wow', 'computer', 'learning', 'fast']

Teks Mentah:
'Wow, the computers at https://example.com are learning so fast! @ML_Lover #AI'

Token Bersih:
['wow', 'computer', 'learning', 'fast']


## Level 3: Mengubah Teks Menjadi Angka (Vectorization/Feature Extraction)

Di sini keajaiban dimulai. Kita mengubah daftar token yang bersih menjadi vektor numerik.

- **Tujuan**: Merepresentasikan teks dalam format matematis yang bisa dipahami model ML.

- **Metode Klasik (Berbasis Frekuensi):**
  - **Bag-of-Words (BoW)**: Metode paling sederhana.
    - **Cara kerja:** Bayangkan kita punya "kantong" (bag) berisi semua kata unik dari vocabulary. Untuk setiap dokumen, kita hitung berapa kali setiap kata dari vocabulary muncul di dokumen itu. Urutan kata diabaikan sama sekali.
    - **Contoh**: Vocabulary: ['aku', 'suka', 'nasi', 'goreng']. Dokumen: "aku suka nasi, aku suka goreng". Vektor BoW-nya: [2, 2, 1, 1].
    - **Kelemahan**: Mengabaikan urutan kata. "aku tidak suka kamu" dan "kamu tidak suka aku" akan memiliki vektor yang sama.
  - **TF-IDF (Term Frequency-Inverse Document Frequency):** Peningkatan dari BoW.
    - **Cara kerja:** Memberikan bobot pada setiap kata. Idenya adalah kata yang sering muncul di satu dokumen (tinggi TF) tetapi jarang muncul di semua dokumen lain dalam corpus (tinggi IDF) adalah kata yang penting dan spesifik untuk dokumen tersebut.
    - **Contoh:** Dalam corpus artikel berita, kata "presiden" mungkin punya skor TF-IDF tinggi di artikel politik, tapi kata "dan" akan punya skor TF-IDF sangat rendah di semua artikel karena muncul di mana-mana.
    - **Kelebihan:** Lebih baik dalam mengidentifikasi kata kunci yang informatif.
  
- **Metode Modern (Berbasis Konteks - Deep Learning):**
  - **Word Embeddings (Contoh: Word2Vec, GloVe, FastText):**
    - **Cara kerja:** Ini adalah revolusi dalam NLP. Setiap kata tidak lagi direpresentasikan oleh satu angka (seperti di BoW), tetapi oleh sebuah vektor padat (misal, 300 angka desimal). Vektor ini dipelajari oleh model neural network dari corpus besar.
    - **Keajaiban:** Kata-kata dengan makna serupa akan memiliki vektor yang berdekatan secara matematis di ruang vektor. Kita bisa melakukan "aljabar kata", seperti: `vektor('Raja') - vektor('Pria') + vektor('Wanita') ≈ vektor('Ratu')`.
    - **Kelebihan:** Menangkap hubungan semantik dan sintaktis antar kata. Ini adalah fondasi dari hampir semua model NLP modern. Lapisan Embedding di Keras yang kita lihat kemarin melakukan hal ini.

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

def vectorize_with_bow(corpus, max_features=5000):
    """
    Mengubah sebuah corpus (daftar dokumen teks) menjadi matriks fitur Bag-of-Words (BoW).
    
    Args:
        corpus (list of str): Daftar string, di mana setiap string adalah satu dokumen.
        max_features (int): Batasi vocabulary dengan hanya mempertimbangkan 'max_features' kata
                              paling umum. Ini sangat berguna untuk mengontrol dimensi.
                              Jika None, semua fitur akan digunakan.
        
    Returns:
        tuple: Berisi:
            - sparse matrix: Matriks fitur BoW.
            - CountVectorizer: Objek vectorizer yang sudah di-fit, bisa digunakan lagi untuk data baru.
    """
    # Inisialisasi CountVectorizer.
    vectorizer = CountVectorizer(stop_words='english', max_features=max_features)
    
    # Fit vectorizer pada corpus dan transform corpus menjadi matriks BoW
    X_bow = vectorizer.fit_transform(corpus)
    
    print(f"Bentuk matriks BoW: {X_bow.shape}")
    print(f"Ukuran vocabulary: {len(vectorizer.get_feature_names_out())}")
    
    return X_bow, vectorizer

# --- Contoh Penggunaan ---
sample_corpus = [
    "This is the first document.",
    "This document is the second document.",
    "And this is the third one.",
    "Is this the first document?",
    "A completely new and different document."
]

# Menggunakan fungsi untuk membuat matriks BoW
bow_matrix, bow_vectorizer = vectorize_with_bow(sample_corpus, max_features=None) # Tidak membatasi fitur untuk contoh ini

# Untuk melihat hasilnya (ini adalah sparse matrix, kita ubah ke array agar mudah dibaca)
print("\nMatriks Bag-of-Words (BoW):\n", bow_matrix.toarray())

# Untuk melihat fitur (kata) apa yang diwakili oleh setiap kolom
print("\nFitur (Vocabulary):\n", bow_vectorizer.get_feature_names_out())

# Contoh interpretasi:
# Baris pertama [0 1 1 0 0 0 1 0] sesuai dengan "This is the first document."
# Kata 'document' ada di indeks 1, 'first' di indeks 2, 'is' tidak ada (karena stopword), 'the' tidak ada (stopword), 'this' tidak ada (stopword).
# Hmm, mari kita lihat fiturnya. Oh, vectorizer secara default men-lowercase semua.
# 'document' ada di indeks 0, 'first' di 1, 'completely' di 2, dst.
# Mari kita cek lagi baris pertama untuk "this is the first document" -> ['document', 'first']
# 'document' (indeks 0) muncul 1x, 'first' (indeks 1) muncul 1x.
# Jadi vektornya akan punya nilai 1 di kolom 0 dan 1.
# Mari kita lihat: bow_matrix.toarray()[0]
# Oh, `get_feature_names_out` sudah diurutkan. Mari kita lihat nama fitur dan posisinya.
# 'document' -> kolom 1, 'first' -> kolom 2.
# Vektor untuk baris pertama: [1, 1, 0, 0, 1, 0, 0] -> document=1, first=1, is=tidak ada, the=tidak ada, this=tidak ada.
# Tunggu, kenapa ada 7 kolom? Mari kita lihat `get_feature_names_out()` -> ['completely', 'different', 'document', 'new', 'one', 'second', 'third']
# Ah, 'first' dan 'this' dan 'is' terhapus oleh 'stop_words=english'.
# Mari kita jalankan ulang tanpa stopwords untuk melihat perilakunya.

print("\n\n--- Menjalankan ulang tanpa stopword removal untuk kejelasan ---")
vectorizer_no_stopwords = CountVectorizer(max_features=None)
X_bow_no_stopwords = vectorizer_no_stopwords.fit_transform(sample_corpus)
print("\nFitur (Vocabulary) tanpa stopword removal:\n", vectorizer_no_stopwords.get_feature_names_out())
print("\nMatriks BoW tanpa stopword removal:\n", X_bow_no_stopwords.toarray())

Bentuk matriks BoW: (5, 5)
Ukuran vocabulary: 5

Matriks Bag-of-Words (BoW):
 [[0 0 1 0 0]
 [0 0 2 0 1]
 [0 0 0 0 0]
 [0 0 1 0 0]
 [1 1 1 1 0]]

Fitur (Vocabulary):
 ['completely' 'different' 'document' 'new' 'second']


--- Menjalankan ulang tanpa stopword removal untuk kejelasan ---

Fitur (Vocabulary) tanpa stopword removal:
 ['and' 'completely' 'different' 'document' 'first' 'is' 'new' 'one'
 'second' 'the' 'third' 'this']

Matriks BoW tanpa stopword removal:
 [[0 0 0 1 1 1 0 0 0 1 0 1]
 [0 0 0 2 0 1 0 0 1 1 0 1]
 [1 0 0 0 0 1 0 1 0 1 1 1]
 [0 0 0 1 1 1 0 0 0 1 0 1]
 [1 1 1 1 0 0 1 0 0 0 0 0]]


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

def vectorize_with_tfidf(corpus):
    """
    Mengubah sebuah corpus (daftar dokumen teks) menjadi matriks fitur TF-IDF.
    
    Args:
        corpus (list of str): Daftar string, di mana setiap string adalah satu dokumen.
        
    Returns:
        tuple: Berisi:
            - sparse matrix: Matriks fitur TF-IDF.
            - TfidfVectorizer: Objek vectorizer yang sudah di-fit, bisa digunakan lagi untuk data baru.
    """
    # Inisialisasi vectorizer.
    vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
    
    # Fit dan transform corpus
    X_tfidf = vectorizer.fit_transform(corpus)
    
    print(f"Bentuk matriks TF-IDF: {X_tfidf.shape}")
    print(f"Ukuran vocabulary: {len(vectorizer.get_feature_names_out())}")
    
    return X_tfidf, vectorizer

# --- Contoh Penggunaan ---
sample_corpus = [
    "This is the first document.",
    "This document is the second document.",
    "And this is the third one.",
    "Is this the first document?"
]
tfidf_matrix, tfidf_vectorizer = vectorize_with_tfidf(sample_corpus)

# Untuk melihat hasilnya (ini adalah sparse matrix)
print("\nMatriks TF-IDF (sparse):\n", tfidf_matrix.toarray())

Bentuk matriks TF-IDF: (4, 2)
Ukuran vocabulary: 2

Matriks TF-IDF (sparse):
 [[1.         0.        ]
 [0.78722298 0.61666846]
 [0.         0.        ]
 [1.         0.        ]]


In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

def vectorize_for_deep_learning(corpus, vocab_size=10000, max_length=256):
    """
    Mengubah corpus teks menjadi sekuens integer yang sudah di-padding, siap untuk model DL.
    
    Args:
        corpus (list of str): Daftar dokumen teks.
        vocab_size (int): Ukuran maksimum vocabulary yang akan digunakan.
        max_length (int): Panjang maksimum setiap sekuens setelah padding.
        
    Returns:
        tuple: Berisi:
            - np.ndarray: Matriks sekuens yang sudah di-padding.
            - Tokenizer: Objek tokenizer yang sudah di-fit.
    """
    # Inisialisasi tokenizer
    tokenizer = Tokenizer(num_words=vocab_size, oov_token="<UNK>")
    
    # Fit tokenizer pada corpus
    tokenizer.fit_on_texts(corpus)
    
    # Ubah teks menjadi sekuens integer
    sequences = tokenizer.texts_to_sequences(corpus)
    
    # Lakukan padding
    padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post')
    
    print(f"Bentuk sekuens setelah padding: {padded_sequences.shape}")
    
    return padded_sequences, tokenizer

# --- Contoh Penggunaan ---
padded_matrix, keras_tokenizer = vectorize_for_deep_learning(sample_corpus, vocab_size=20, max_length=10)

print("\nMatriks sekuens setelah padding:\n", padded_matrix)

# Untuk melihat word index yang dipelajari tokenizer
print("\nWord Index:\n", keras_tokenizer.word_index)

Bentuk sekuens setelah padding: (4, 10)

Matriks sekuens setelah padding:
 [[ 2  3  4  6  5  0  0  0  0  0]
 [ 2  5  3  4  7  5  0  0  0  0]
 [ 8  2  3  4  9 10  0  0  0  0]
 [ 3  2  4  6  5  0  0  0  0  0]]

Word Index:
 {'<UNK>': 1, 'this': 2, 'is': 3, 'the': 4, 'document': 5, 'first': 6, 'second': 7, 'and': 8, 'third': 9, 'one': 10}


## Level 4: Membangun Model (Tasks & Architectures)

Setelah teks menjadi vektor, kita bisa memasukkannya ke model ML untuk melakukan berbagai tugas.

**Tugas NLP Umum:**
- **Klasifikasi Teks**: Memberi label pada sebuah dokumen. (Contoh: Klasifikasi Sentimen, Deteksi Spam, Kategorisasi Topik Berita).
- **Named Entity Recognition (NER)**: Mengidentifikasi dan melabeli "entitas" dalam teks. (Contoh: "Apple [ORG] didirikan oleh Steve Jobs [PER] di Cupertino [LOC]").
- **Machine Translation**: Menerjemahkan teks dari satu bahasa ke bahasa lain.
- **Summarization**: Membuat ringkasan singkat dari teks yang panjang.
- **Question Answering**: Memberikan jawaban berdasarkan konteks teks yang diberikan.

**Arsitektur Model:**
- **Model Klasik**: Naive Bayes, Logistic Regression, SVM (bekerja baik dengan fitur TF-IDF).
- **Recurrent Neural Networks (RNN)**: Didesain khusus untuk data berurutan (seperti teks). Ia memiliki "memori" yang memproses kata satu per satu sambil mengingat apa yang telah ia lihat sebelumnya.
    - **LSTM (Long Short-Term Memory) & GRU (Gated Recurrent Unit)**: Varian RNN yang jauh lebih baik dalam menangani "memori jangka panjang", mampu menghubungkan kata di awal kalimat dengan kata di akhir kalimat. Sangat populer untuk klasifikasi teks dan NER.
- **Convolutional Neural Networks (CNN)**: Awalnya untuk gambar, tetapi juga bisa digunakan untuk NLP. Ia bekerja dengan melihat "kelompok kata" (n-grams) dan bisa sangat cepat dan efektif untuk klasifikasi.
- **Transformers (Revolusi Terbaru)**: Arsitektur yang diperkenalkan di paper "Attention Is All You Need" (2017). Ia tidak memproses kata secara berurutan, melainkan melihat semua kata sekaligus dan menggunakan mekanisme "Attention" untuk menimbang seberapa penting setiap kata lain bagi kata yang sedang diproses.
    - **Model-model seperti BERT, GPT (Generative Pre-trained Transformer), T5** adalah contoh model berbasis Transformer. Mereka adalah state-of-the-art untuk hampir semua tugas NLP saat ini. Mereka biasanya sudah di-pre-trained pada data teks yang masif (seluruh internet) dan bisa kita fine-tune untuk tugas spesifik kita.
